In [1]:
import redshift_connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [80]:
def fetch_OHLCV_df_from_redshift(asset_id_base, asset_id_quote, exchange_id):
    # Connect to Redshift cluster containing price data
    with redshift_connector.connect(
        host = 'project-poseidon.cpsnf8brapsd.us-west-2.redshift.amazonaws.com',
        database = 'token_price',
        user = 'administrator',
        password = 'Free2play2'
    ) as conn:
        with conn.cursor() as cursor:
            # Query to fetch OHLCV data for a token & exchange of interest
            title = asset_id_base + '-' + asset_id_quote
            query = """
            SELECT 
                time_period_end,
                price_open,
                price_high,
                price_low,
                price_close,
                volume_traded
            FROM token_price.coinapi.price_data_1h
            WHERE 
                asset_id_base = '{}' AND
                asset_id_quote = '{}' AND
                exchange_id = '{}'
            ORDER BY time_period_start ASC
            """.format(asset_id_base, asset_id_quote, 
                        exchange_id)

            # Execute query on Redshift and return result as a DataFrame
            cursor.execute(query)

            df = cursor.fetch_dataframe().rename({'time_period_end':'Date'}, axis = 1).set_index('Date').astype(float)
            return df

,price_close
Date,
2011-09-13 14:00:00,6.00
2011-09-13 15:00:00,5.76
2011-09-13 16:00:00,5.65
2011-09-13 20:00:00,5.97
2011-09-14 08:00:00,5.62
...,...
2023-08-04 00:00:00,29176.00
2023-08-04 01:00:00,29213.00
2023-08-04 02:00:00,29180.00


In [ ]:
ohlcv = fetch_OHLCV_df_from_redshift('BTC', 'USD', 'COINBASE')
order_book_snapshot = pd.read_csv('order_book_snapshot.csv')
blockchin_metrics = ...